In [2]:
import torchaudio
import allin1
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from pydub import AudioSegment
import numpy as np

In [ ]:
# You can analyze a single file:
result = allin1.analyze('.//assets//empty Brain.mp3')

# Or multiple files:
#results = allin1.analyze(['your_audio_file1.wav', 'your_audio_file2.mp3'])

# Generate Audio from sample

In [3]:
# Load the pretrained model
model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=8)  # generate 8 seconds.

# Generate unconditional audio samples
wav_unconditional = model.generate_unconditional(4)  # generates 4 unconditional audio samples

# Generate audio samples based on descriptions
descriptions = ['breakcore', 'IDM', 'hyperpop']
wav_descriptions = model.generate(descriptions)  # generates 3 samples.

# Load melody and generate audio samples with chroma
melody, sr = torchaudio.load('.//assets//trimmed_audiofile.mp3')
wav_chroma = model.generate_with_chroma(descriptions, melody[None].expand(3, -1, -1), sr)

e:\dev\audiocraft\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


# Stitch audio segments

In [4]:
# Function to convert tensor to AudioSegment
def tensor_to_audiosegment(tensor, sample_rate):
    audio_array = tensor.cpu().numpy()
    audio_array = (audio_array * 32767).astype(np.int16)  # Convert to int16
    return AudioSegment(
        audio_array.tobytes(),
        frame_rate=sample_rate,
        sample_width=audio_array.dtype.itemsize,    
        channels=1
    )



In [5]:
# Convert generated audio tensors to AudioSegment
audio_segments = []



In [6]:

# Process chroma-based audio samples
for wav in wav_chroma:
    audio_segments.append(tensor_to_audiosegment(wav, model.sample_rate))

# Concatenate all audio segments
blended_audio = sum(audio_segments)

# Export the final blended audio
blended_audio.export("blended_output.wav", format="wav")

<_io.BufferedRandom name='blended_output.wav'>

In [7]:
# Process unconditional audio samples
for wav in wav_unconditional:
    audio_segments.append(tensor_to_audiosegment(wav, model.sample_rate))

# Process description-based audio samples
for wav in wav_descriptions:
    audio_segments.append(tensor_to_audiosegment(wav, model.sample_rate))